In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Chew-Mandelstam S-wave and dispersion integrals
TR-003
^^^
Section {ref}`report/003:S-wave` has been implemented in [ampform#265](https://github.com/ComPWA/ampform/issues/265).
+++
_To be implemented_
:::
::::

# Chew-Mandelstam

<!-- cspell:ignore dasharray dashoffset displaystyle displaystyledisplaystyle linecap linejoin ndim orthopy xlink xtick ytick -->
<!-- cspell:ignore epsabs epsrel pinf pyfunc -->

In [ ]:
%pip install -q ampform==0.13.3 black==22.3.0 matplotlib==3.5.1 mpl-interactions==0.20.2 ndim==0.1.7 numpy==1.22.3 qrules==0.9.7 quadpy==0.16.15 scipy==1.8.0 sympy==1.10.1 x21==0.2.16

This report is an attempt formulate the Chew-Mandelstam function described in {pdg-review}`2021; Resonances; pp.13` (Section 50.3.5) with [SymPy](https://docs.sympy.org), so that it can be implemented in [AmpForm](https://ampform.rtfd.io).

In [ ]:
%matplotlib widget

In [ ]:
import inspect
import warnings
from functools import partial

import black
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np
import qrules
import quadpy
import symplot
import sympy as sp
from ampform.dynamics import (
    BlattWeisskopfSquared,
    BreakupMomentumSquared,
    ComplexSqrt,
    PhaseSpaceFactor,
    PhaseSpaceFactorComplex,
)
from IPython.display import Math

warnings.filterwarnings("ignore")
PDG = qrules.load_pdg()

## S-wave

As can be seen in Eq. (50.40) on {pdg-review}`2021; Resonances; p.13`, the Chew-Mandelstam function $\Sigma_a$ for a particle $a$ decaying to particles $1, 2$ has a simple form for angular momentum $L=0$ ($S$-wave):

:::{math}
:class: full-width
:label: chew-mandelstam
\Sigma_a(s) = \frac{1}{16\pi^2}
\left[
 \frac{2q_a}{\sqrt{s}}
 \log\frac{m_1^2+m_2^2-s+2\sqrt{s}q_a}{2m_1m_2}
 - \left(m_1^2-m_2^2\right)
 \left(\frac{1}{s}-\frac{1}{(m_1+m_2)^2}\right)
 \log\frac{m_1}{m_2}
\right]
:::

The only question is how to deal with negative values for the squared break-up momentum $q_a^2$. Here, we will use AmpForm's {class}`~ampform.sympy.math.ComplexSqrt`:

In [ ]:
q_squared_symbol = sp.Symbol("q_a^{2}", real=True)
q_a_expr = ComplexSqrt(q_squared_symbol)
Math(f"q_a = {sp.latex(q_a_expr)} = {sp.latex(q_a_expr.evaluate())}")

<IPython.core.display.Math object>

In [ ]:
def breakup_momentum(s, m1, m2):
    q_squared = BreakupMomentumSquared(s, m1, m2)
    return ComplexSqrt(q_squared)


def chew_mandelstam_s_wave(s, m1, m2):
    # evaluate=False in order to keep same style as PDG
    q = breakup_momentum(s, m1, m2)
    left_term = sp.Mul(
        2 * q / sp.sqrt(s),
        sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2)),
        evaluate=False,
    )
    right_term = (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
    return sp.Mul(
        1 / (16 * sp.pi**2),
        left_term - right_term,
        evaluate=False,
    )

To check whether this implementation is correct, let's plug some {class}`~sympy.core.symbol.Symbol`s into this function and compare it to Eq. (50.40) on {pdg-review}`2021; Resonances; p.13`:

In [ ]:
s, m1, m2 = sp.symbols("s m1 m2", real=True)
chew_mandelstam_s_wave_expr = chew_mandelstam_s_wave(s, m1, m2)
chew_mandelstam_s_wave_expr

(1/(16*pi**2))*((2*ComplexSqrt(BreakupMomentumSquared(s, m1, m2))/sqrt(s))*log((m1**2 + m2**2 + 2*sqrt(s)*ComplexSqrt(BreakupMomentumSquared(s, m1, m2)) - s)/(2*m1*m2)) - (m1**2 - m2**2)*(-1/(m1 + m2)**2 + 1/s)*log(m1/m2))

It should be noted that this equation is not well-defined along the real axis, that is, for $\mathrm{Im}(s) = 0$. For this reason, we split $s$ into a real part $s'$ with a small imaginary offset (the PDG indicates this with $s+0i$). We parametrized this imaginary offset with $\epsilon$, and for the interactive plot, we do so with a power of $10$:

In [ ]:
epsilon = sp.Symbol("epsilon", positive=True)
s_prime = sp.Symbol(R"s^{\prime}", real=True)
s_plus = s_prime + sp.I * sp.Pow(10, -epsilon)

In [ ]:
Math(Rf"{sp.latex(s)} \to {sp.latex(s_plus)}")

<IPython.core.display.Math object>

We are now ready to use [`mpl_interactions`](https://mpl-interactions.rtfd.io) and AmpForm's {mod}`symplot` to visualize this function:

In [ ]:
chew_mandelstam_s_wave_prime = chew_mandelstam_s_wave_expr.subs(s, s_plus)
np_chew_mandelstam_s_wave, sliders = symplot.prepare_sliders(
    expression=chew_mandelstam_s_wave_prime.doit(),
    plot_symbol=s_prime,
)
np_phase_space_factor = sp.lambdify(
    args=(s_prime, m1, m2, epsilon),
    expr=PhaseSpaceFactorComplex(s_plus, m1, m2).doit(),
    modules="numpy",
)

As starting values for the interactive plot, we assume $\pi\eta$ scattering (just like in the PDG section) and use their masses as values for $m_1$ and $m_1$, respectively.

In [ ]:
s_min, s_max = -0.15, 1.4
m1_val = PDG["pi0"].mass
m2_val = PDG["eta"].mass

plot_domain = np.linspace(s_min, s_max, 500)
sliders.set_ranges(
    m1=(0, 2, 200),
    m2=(0, 2, 200),
    epsilon=(1, 12),
)
sliders.set_values(
    m1=m1_val,
    m2=m2_val,
    epsilon=4,
)

For comparison, we plot the Chew-Mandelstam function for $S$-waves next to AmpForm's {class}`~ampform.dynamics.phasespace.PhaseSpaceFactorComplex`. Have a look at the resulting plots and compare to Figure 50.4 on {pdg-review}`2021; Resonances; p.12`.

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(11, 4.5), tight_layout=True)
ax1, ax2 = axes
for ax in axes:
    ax.axhline(0, linewidth=0.5, c="black")

real_style = {"label": "Real part", "c": "black", "linestyle": "dashed"}
imag_style = {"label": "Imag part", "c": "red"}
threshold_style = {"label": R"$s_\mathrm{thr}$", "c": "grey", "linewidth": 0.5}

ylim = (-1, +1)
y_factor = 16 * np.pi
controls = iplt.axvline(
    lambda *args, **kwargs: (kwargs["m1"] + kwargs["m2"]) ** 2,
    **sliders,
    ax=ax1,
    **threshold_style,
)
iplt.axvline(
    lambda *args, **kwargs: (kwargs["m1"] + kwargs["m2"]) ** 2,
    controls=controls,
    ax=ax2,
    **threshold_style,
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: (
        y_factor * 1j * np_phase_space_factor(*args, **kwargs)
    ).real,
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax1,
    **real_style,
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: (
        y_factor * 1j * np_phase_space_factor(*args, **kwargs)
    ).imag,
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax1,
    **imag_style,
)

iplt.plot(
    plot_domain,
    lambda *args, **kwargs: y_factor
    * np_chew_mandelstam_s_wave(*args, **kwargs).real,
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax2,
    **real_style,
)
iplt.plot(
    plot_domain,
    lambda *args, **kwargs: y_factor
    * np_chew_mandelstam_s_wave(*args, **kwargs).imag,
    controls=controls,
    ylim=ylim,
    alpha=0.7,
    ax=ax2,
    **imag_style,
)

for ax in axes:
    ax.legend(loc="lower right")
    ax.set_xticks(np.arange(0, 1.21, 0.3))
    ax.set_yticks(np.arange(-1, 1.1, 0.5))
    ax.set_xlabel("$s$ (GeV$^2$)")

ax1.set_ylabel(R"$16\pi \; i\rho(s)$")
ax2.set_ylabel(R"$16\pi \; \Sigma(s)$")
ax1.set_title(R"Complex phase space factor $\rho$")
ax2.set_title("Chew-Mandelstam $S$-wave ($L=0$)")
plt.show()

In [ ]:
plt.savefig("003-chew-mandelstam-s-wave.svg")

```{figure} https://user-images.githubusercontent.com/29308176/164984924-764a9558-6afd-46a9-8f24-8cc92ce1bc49.svg
:class: full-width
```

## General dispersion integral

For higher angular momenta, the PDG notes that one has to compute the dispersion integral given by Eq. (50.41) on {pdg-review}`2021; Resonances; p.13`:

$$
\Sigma_a(s+0i) =
    \frac{s-s_{\mathrm{thr}_a}}{\pi}
    \int^\infty_{s_{\mathrm{thr}_a}} \frac{
        \rho_a(s')n_a^2(s')
    }{
        (s' - s_{\mathrm{thr}_a})(s'-s-i0)
    }
    \mathop{}\!\mathrm{d}s'
$$ (dispersion-integral)

Equation {eq}`chew-mandelstam` is the analytic solution for $L=0$.

From Equations (50.26-27) on {pdg-review}`2021; Resonances; p.9`, it can be deduced that the function $n_a^2$ is the same as AmpForm's {class}`~ampform.dynamics.BlattWeisskopfSquared` (note that this function is normalized, whereas the PDG's $F_j$ function has $1$ in the nominator). Furthermore, the PDG seems to suggest that $z = q_a/q_0$, but this is an unconventional choice and is probably a mistake. For this reason, we simply use {class}`~ampform.dynamics.BlattWeisskopfSquared` for the definition of $n_a^2$:

In [ ]:
def na2(s, m1, m2, L, q0):
    q_squared = BreakupMomentumSquared(s, m1, m2)
    return BlattWeisskopfSquared(
        z=q_squared / (q0**2),
        angular_momentum=L,
    )

For $\rho_a$, we use AmpForm's {class}`~ampform.dynamics.phasespace.PhaseSpaceFactor`:

In [ ]:
q0 = sp.Symbol("q0", real=True)
L = sp.Symbol("L", integer=True, positive=True)
s_thr = (m1 + m2) ** 2
integrand = (PhaseSpaceFactor(s_prime, m1, m2) * na2(s_prime, m1, m2, L, q0)) / (
    (s_prime - s_thr) * (s_prime - s - epsilon * sp.I)
)
integrand

BlattWeisskopfSquared(L, BreakupMomentumSquared(s^{\prime}, m1, m2)/q0**2)*PhaseSpaceFactor(s^{\prime}, m1, m2)/((s^{\prime} - (m1 + m2)**2)*(-I*epsilon - s + s^{\prime}))

Next, we {func}`~sympy.utilities.lambdify.lambdify` this integrand to a {mod}`numpy` expression so that we can integrate it efficiently:

In [ ]:
np_integrand = sp.lambdify(
    args=(s_prime, s, L, epsilon, m1, m2, q0),
    expr=integrand.doit(),
    modules="numpy",
)

As discussed in [TR-016](016.ipynb), {func}`scipy.integrate.quad` cannot integrate over complex-valued functions, while [`quadpy`](https://github.com/sigma-py/quadpy) runs into trouble with vectorized input to the integrand. The following function, from {ref}`report/016:Vectorized input` offers a quick solution:

In [ ]:
@np.vectorize
def vectorized_quad(func, a, b, **func_kwargs):
    values, _ = quadpy.quad(partial(func, **func_kwargs), a, b)
    return values

:::{note}

Integrals can be expressed with SymPy, with some caveats. See {ref}`report/016:SymPy integral`.

:::

Now, for comparison, we compute this integral for a few values of $L>0$:

In [ ]:
s_domain = np.linspace(s_min, s_max, num=50)
max_L = 3
l_values = list(range(1, max_L + 1))
print("Computing for L ∈", l_values)

Computing for L ∈ [1, 2, 3]


It is handy to store the resulting values of each dispersion integral in a {obj}`dict` with $L$ as keys:

```{autolink-skip}
```

In [ ]:
%%time
s_thr_val = float(s_thr.subs({m1: m1_val, m2: m2_val}))
integral_values = {
    l_val: vectorized_quad(
        np_integrand,
        a=s_thr_val,
        b=np.inf,
        s=s_domain,
        L=l_val,
        epsilon=1e-3,
        m1=m1_val,
        m2=m2_val,
        q0=1.0,
    )
    for l_val in l_values
}

CPU times: user 2.61 s, sys: 7.39 ms, total: 2.62 s
Wall time: 2.62 s


Finally, as can be seen from Eq. {eq}`dispersion-integral`, the resulting values from the integral have to be shifted with a factor $\frac{s-s_{\mathrm{thr}_a}}{\pi}$ to get $\Sigma_a$. We also scale the values with $16\pi$ so that it can be compared with the plot generated in {ref}`report/003:S-wave`.

In [ ]:
sigma = {
    l_val: (s_domain - s_thr_val) / np.pi * integral_values[l_val]
    for l_val in l_values
}
sigma_scaled = {l_val: 16 * np.pi * sigma[l_val] for l_val in l_values}

In [ ]:
fig, axes = plt.subplots(
    nrows=len(l_values),
    sharex=True,
    figsize=(5, 2.5 * len(l_values)),
    tight_layout=True,
)
fig.suptitle(f"Dispersion integrals for $m_1={m1_val:.2f}, m_2={m2_val:.2f}$")
for ax, l_val in zip(axes, l_values):
    ax.axhline(0, linewidth=0.5, c="black")
    ax.axvline(s_thr_val, **threshold_style)
    ax.plot(s_domain, sigma_scaled[l_val].real, **real_style)
    ax.plot(s_domain, sigma_scaled[l_val].imag, **imag_style)
    ax.set_title(f"$L = {l_val}$")
    ax.set_ylabel(R"$16\pi \; \Sigma(s)$")
axes[-1].set_xlabel("$s$ (GeV$^2$)")
axes[0].legend()
plt.show()

In [ ]:
plt.savefig("003-chew-mandelstam-l-non-zero.svg")

![Chew-Mandelstam for higher angular momenta](https://user-images.githubusercontent.com/29308176/164985017-7600941e-0481-4282-8d9b-0680f720e6ef.svg)

:::{note}

In {ref}`report/003:SymPy expressions` we'll see that the dispersion integral indeed reproduces the same shape as the analytic expression from {ref}`report/003:S-wave`.

:::

## SymPy expressions

In the following, we attempt to implement Equation {eq}`dispersion-integral` using {ref}`report/016:SymPy integral`.

In [ ]:
from sympy.printing.pycode import _unpack_integral_limits


class UnevaluatableIntegral(sp.Integral):
    abs_tolerance = 1e-5
    rel_tolerance = 1e-5
    limit = 50

    def doit(self, **hints):
        args = [arg.doit(**hints) for arg in self.args]
        return self.func(*args)

    def _numpycode(self, printer, *args):
        integration_vars, limits = _unpack_integral_limits(self)
        if len(limits) != 1:
            msg = f"Cannot handle {len(limits)}-dimensional integrals"
            raise ValueError(msg)
        integrate = "quadpy_quad"
        printer.module_imports["quadpy"].update({f"quad as {integrate}"})
        limit_str = "{}, {}".format(*tuple(map(printer._print, limits[0])))
        args = ", ".join(map(printer._print, integration_vars))
        expr = printer._print(self.args[0])
        return (
            f"{integrate}(lambda {args}: {expr}, {limit_str},"
            f" epsabs={self.abs_tolerance}, epsrel={self.abs_tolerance},"
            f" limit={self.limit})[0]"
        )

In [ ]:
def dispersion_integral(
    s,
    m1,
    m2,
    angular_momentum,
    meson_radius=1,
    s_prime=sp.Symbol("x", real=True),
    epsilon=sp.Symbol("epsilon", positive=True),
):
    s_thr = (m1 + m2) ** 2
    q_squared = BreakupMomentumSquared(s_prime, m1, m2)
    ff_squared = BlattWeisskopfSquared(
        angular_momentum=L, z=q_squared * meson_radius**2
    )
    phsp_factor = PhaseSpaceFactor(s_prime, m1, m2)
    return sp.Mul(
        (s - s_thr) / sp.pi,
        UnevaluatableIntegral(
            (phsp_factor * ff_squared)
            / (s_prime - s_thr)
            / (s_prime - s - sp.I * epsilon),
            (s_prime, s_thr, sp.oo),
        ),
        evaluate=False,
    )


x = sp.Symbol("x", real=True)
integral_expr = dispersion_integral(s, m1, m2, angular_momentum=L, s_prime=x)
integral_expr

((s - (m1 + m2)**2)/pi)*Integral(BlattWeisskopfSquared(L, BreakupMomentumSquared(x, m1, m2))*PhaseSpaceFactor(x, m1, m2)/((x - (m1 + m2)**2)*(-I*epsilon - s + x)), (x, (m1 + m2)**2, oo))

:::{warning}

We have to keep track of the integration variable ($s'$ in Equation {eq}`dispersion-integral`), so that we don't run into trouble if we use {func}`~sympy.utilities.lambdify.lambdify` with common sub-expressions. The problem is that the integration variable _should not_ be extracted as a common sub-expression, otherwise the lambdified [`quadpy.quad()`](https://github.com/sigma-py/quadpy) expression cannot handle vectorized input.

:::

To keep the function under the integral simple, we substitute angular momentum $L$ with a definite value before we lambdify:

In [ ]:
UnevaluatableIntegral.abs_tolerance = 1e-4
UnevaluatableIntegral.rel_tolerance = 1e-4
integral_func_s_wave = sp.lambdify(
    [s, m1, m2, epsilon],
    integral_expr.subs(L, 0).doit(),
    # integration symbol should not be extracted as common sub-expression!
    cse=partial(sp.cse, ignore=[x], list=False),
)
integral_func_s_wave = np.vectorize(integral_func_s_wave)

integral_func_p_wave = sp.lambdify(
    [s, m1, m2, epsilon],
    integral_expr.subs(L, 1).doit(),
    cse=partial(sp.cse, ignore=[x], list=False),
)
integral_func_p_wave = np.vectorize(integral_func_p_wave)

In [ ]:
src = inspect.getsource(integral_func_s_wave.pyfunc)
src = black.format_str(src, mode=black.FileMode())
print(src)

def _lambdifygenerated(s, m1, m2, epsilon):
    x0 = pi ** (-1.0)
    x1 = (m1 + m2) ** 2
    x2 = -x1
    return (
        x0
        * (s + x2)
        * quadpy_quad(
            lambda x: (1 / 16)
            * x0
            * sqrt((x + x2) * (x - (m1 - m2) ** 2) / x)
            / (sqrt(x) * (x + x2) * (-1j * epsilon - s + x)),
            x1,
            PINF,
            epsabs=0.0001,
            epsrel=0.0001,
            limit=50,
        )[0]
    )



```{autolink-skip}
```

In [ ]:
s_values = np.linspace(-0.15, 1.4, num=200)
%time s_wave_values = integral_func_s_wave(s_values, m1_val, m2_val, epsilon=1e-5)
%time p_wave_values = integral_func_p_wave(s_values, m1_val, m2_val, epsilon=1e-5)

CPU times: user 5.13 s, sys: 0 ns, total: 5.13 s
Wall time: 5.13 s
CPU times: user 2.41 s, sys: 0 ns, total: 2.41 s
Wall time: 2.41 s


Note that the dispersion integral for $L=0$ indeed reproduces the same shape as in {ref}`report/003:S-wave`!

In [ ]:
s_wave_values *= 16 * np.pi
p_wave_values *= 16 * np.pi

s_values = np.linspace(-0.15, 1.4, num=200)
fig, axes = plt.subplots(nrows=2, figsize=(6, 7), sharex=True)
ax1, ax2 = axes
fig.suptitle(
    f"Symbolic dispersion integrals for $m_1={m1_val:.2f}, m_2={m2_val:.2f}$"
)
for ax in axes:
    ax.axhline(0, linewidth=0.5, c="black")
    ax.axvline(s_thr_val, **threshold_style)
    ax.set_title(f"$L = {l_val}$")
    ax.set_ylabel(R"$16\pi \; \Sigma(s)$")
axes[-1].set_xlabel("$s$ (GeV$^2$)")

ax1.set_title("$S$-wave ($L=0$)")
ax1.plot(s_values, s_wave_values.real, **real_style)
ax1.plot(s_values, s_wave_values.imag, **imag_style)

ax2.set_title("$P$-wave ($L=1$)")
ax2.plot(s_values, p_wave_values.real, **real_style)
ax2.plot(s_values, p_wave_values.imag, **imag_style)

ax1.legend()
fig.tight_layout()
plt.show()

In [ ]:
plt.savefig("003-symbolic-chew-mandelstam.svg")

![Symbolic Chew-Mandelstam plots](https://user-images.githubusercontent.com/29308176/164984984-dfe73d4c-e604-4d06-b4e1-50be117a57e3.svg)